# Machine Learning (Regression)
* Eğitmek için gerekli veriyi çekelim
* Encoding
* Training
* Prediction

## Veri kümesine bağlanalım

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./dataForTraining.csv")
df.head(3)

,Gün,Lokasyon,Sıcaklık,Broşür,Fiyat,Satış
0,Monday,Park,22,90.0,2.5,137
1,Tuesday,Park,23,90.0,5.0,139
2,Wednesday,Park,22,104.0,5.0,148


## Encoding ve Scaling
* Gün için **Label (Ordinal) Encoding**
    * `df.Gün.asype("category")` sonra `df.Gün.cat.codes`
    * `sklearn.preprocessing` altında `OrdinalEncoder`
    * Günler cycle olduğu için açısal değerler üretelim. **sin veya cos** gibi.
* Lokasyon için **OneHotEncoding**
    * `pd.get_dummies(df,columns=["Lokasyon"])`
    * `sklearn.preprocessing` altında `OneHotEncoder`
* Sıcaklık için **MinMaxScaler**
    * `sklearn.preprocessing` altında `MinMaxScaler`
* Broşür ve Fiyat için **StandartScaler**
    * `sklearn.preprocessing` altında `StandardScaler`

### Pandas ile OneHotEncoding ve LabelEncoding
* sadece alternatifi görelim. Zaten hepsini sklearn ile yapacağız.

In [3]:
#Label Encoding
#df.astype({"Gün":"category"}).Gün.cat.codes
df.Gün.astype("category").cat.codes

#OneHotEncoding
pd.get_dummies(df,columns=["Lokasyon"])

#
pass


## Sklearn ile ColumnTransformer kullanarak Encoding ve Scaling
* Encoder
* ColumnTransformer
    * Tüm dönüşümleri bir arada tutar.
    * ColumnTransformer belirtilen kolonlara belirtilen Encoderları uygular. Custom bir fonksiyon **FunctionTransformer** ile verilebilir.
    * Kolon adı veya sırası liste şeklinde belirilir. Ad verilirse çıkarım yaparken de bu ad beklenir.
    * Belirtilen kolonlar df olarak Encodera gönderilir. Belirilen kolon yerine dönüşüm yapılır.

In [4]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,MinMaxScaler,StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer

In [5]:
def fnk_CustomTransformer(dfg):
    #0.adım
    dfg = dfg.astype("category")

    #1.adım numara verelim
    #dfg.Gün = dfg.Gün.cat.codes

    #2.adım acısal değer üretelim
    dfg.Gün = np.sin((dfg.Gün.cat.codes+1) * (2 * np.pi /  7))
    

    #3.
    # dfg['GünSin'] = np.sin((dfg.Gün.cat.codes+1) * (2 * np.pi /  7)) 
    # dfg['GünCos'] = np.cos((dfg.Gün.cat.codes+1) * (2 * np.pi /  7)) 
    # dfg = dfg.drop(columns=["Gün"])
    return dfg

In [6]:
ct_EncodingveScaling = ColumnTransformer(transformers=[
                                                        #Encoder
                                                        #("ord",OrdinalEncoder(),["Gün"]), #döngü olduğu için açısal değerle de deneyeceğiz.
                                                        ("ord",FunctionTransformer(fnk_CustomTransformer),["Gün"]),
                                                        
                                                        ("ohe",OneHotEncoder(),["Lokasyon"]),
                                                        
                                                        #scaler
                                                        ("mms",MinMaxScaler(),["Sıcaklık"]),
                                                        ("sts",StandardScaler(),["Broşür","Fiyat"])
                                                      ])

#
#[:3] #dizi döndüğü için slicer ile 3 satır çağıralım sadece
pd.DataFrame(ct_EncodingveScaling.fit_transform(df)).head(3)

,0,1,2,3,4,5
0,9.749279e-01,1.0,0.0,0.000000,-0.982972,-1.950060
1,-7.818315e-01,1.0,0.0,0.142857,-0.982972,-0.885767
2,-2.449294e-16,1.0,0.0,0.000000,-0.264975,-0.885767


## Pipeline inşa edelim
* Bir iş akışı oluşturma imkanı veriyor.
* Pipeline steps:
    * Transformer nesnesi: ColumnTransformer
    * Estimator nesnesi: LinearRegression veya DecisionTreeRegressor

In [7]:
from sklearn.pipeline import Pipeline

#
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [8]:
#1. model eğitimi için
pl_LR = Pipeline(steps=[
                            ('preprocessing',ct_EncodingveScaling),
                            ('lr',LinearRegression(fit_intercept=True))
                        ])

#2. model eğitimi için
pl_DTR = Pipeline(steps=[
                            ('preprocessing',ct_EncodingveScaling),
                            ('dtr',DecisionTreeRegressor(criterion="squared_error")) #mean squared error
                        ])

## Veriyi Train ve Test olarak ayıralım
* y = f(X) bulmaya çalışıyoruz. 
* X eğitmek için kullanılan kolonlar y ise tahmin edilecek kolon

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
#y = f(X)
X = df.loc[:,df.columns != "Satış"]
y = df.Satış

#split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=0)

#
print(f"X_train: {len(X_train)}\nX_test:{len(X_test)}")

X_train: 21
X_test:10


## Pipeline tetikleyip modelleri eğitelim

In [11]:
#%time
#model_LR = pl_LR.fit(X_train,y_train)
#model_DTR = pl_DTR.fit(X_train,y_train)

#
#birden fazla model eğitilecek ise döngü iyi olabilir.
egitilmisModeller = []
for pl in [pl_LR, pl_DTR]:
    egitilmisModeller.append(pl.fit(X_train, y_train))


model_LR = egitilmisModeller[0]
model_DTR = egitilmisModeller[1]

## Eğitilmiş model ile tahminler yapalım

In [12]:
y_pred_lr = model_LR.predict(X_test)
y_pred_dtr = model_DTR.predict(X_test)

#sonuçları df olarak gösterelim
pd.DataFrame(data = zip(y_test,y_pred_lr,y_pred_dtr),
             columns=["y_test","y_pred_lr","y_pred_dtr"]).head()

,y_test,y_pred_lr,y_pred_dtr
0,148,146.812416,175.0
1,139,143.626345,127.0
2,163,162.118576,168.0
3,203,195.246092,196.0
4,145,147.572658,147.0


## Evaluate Edelim

In [13]:
from sklearn.metrics import r2_score, mean_squared_error

In [14]:
print(f"LR için RMSE {mean_squared_error(y_test,y_pred_lr)**0.5}")
print(f"DTR için RMSE {mean_squared_error(y_test,y_pred_dtr)**0.5}")
print()

print(f"LR için R2 {r2_score(y_test,y_pred_lr)**0.5}")
print(f"DTR için R2 {r2_score(y_test,y_pred_dtr)**0.5}")
#LR daha iyi sonuç verdi.

LR için RMSE 6.181592244816543
DTR için RMSE 13.820274961085254

LR için R2 0.9620725489253659
DTR için R2 0.7924869712019375


### Model hakkında bilgi alalım

In [15]:
model_LR.feature_names_in_
model_LR.named_steps

#y = wX + b
model_LR._final_estimator #modelin kendisi
model_LR._final_estimator.coef_ # eğitim yani w katsayıları
model_LR._final_estimator.intercept_ #y eksenini kestiği nokta yani b

print(f"y = {model_LR._final_estimator.coef_}*X + {model_LR._final_estimator.intercept_} ")

y = [-3.82158465  0.96242845 -0.96242845 30.23511357 15.0735277  -7.63869363]*X + 138.2329021340975 


 ## Modeli Kaydedelim, diskten okuyalım ve tahmin yapalım

In [16]:
import joblib

#modeli kaydedelim.
joblib.dump(model_LR,"./model_LR.pkl")

['./model_LR.pkl']

In [17]:
#diskten okuyalım
model = joblib.load("./model_LR.pkl")

In [18]:
df.head()

,Gün,Lokasyon,Sıcaklık,Broşür,Fiyat,Satış
0,Monday,Park,22,90.0,2.5,137
1,Tuesday,Park,23,90.0,5.0,139
2,Wednesday,Park,22,104.0,5.0,148
3,Thursday,Plaj,25,98.0,5.0,175
4,Friday,Plaj,26,135.0,5.0,196


In [19]:
#tahmin edilecekler
tahminEdilecekler = pd.DataFrame(data = [
                                            ["Monday","Park",22,90,2.5], #137
                                            ["Monday","Park",22,100,5],
                                            ["Monday","Plaj",22,100,5]
                                        ],
                                columns=["Gün","Lokasyon","Sıcaklık","Broşür","Fiyat"]) 


#
model.predict(tahminEdilecekler)

array([136.31012803, 136.80754493, 134.88268802])